In [35]:
""" 
Análisis sobre la relación entre el crecimiento del número de ataques de tiburón y los océanos con mayor
masa de plástico.

Este es el ranking de los océanos con mayor contaminación: 
https://ourworldindata.org/grapher/surface-plastic-mass-by-ocean
"""


' \nAnálisis sobre la relación entre el crecimiento del número de ataques de tiburón y los océanos con mayor\nmasa de plástico.\n\nEste es el ranking de los océanos con mayor contaminación: \nhttps://ourworldindata.org/grapher/surface-plastic-mass-by-ocean\n'

In [36]:
# Importo pandas para trabajar con dataframes, y numpy para posibles operaciones numéricas con los datos.

import pandas as pd
import numpy as np

In [37]:
# Leo el dataset. Tengo problemas al leerlo porque la codificación es Utt8, incompatible "con mi ordenador". 
# Por tanto, cambio la codificación a ISO-8859-1.

dataset = pd.read_csv('GSAF5.csv', encoding = "ISO-8859-1")

In [104]:
# Head para primera visualización de los datos

display(dataset.head())

,Case Number,Year,Country,Area
0,2016.09.18.c,2016,USA,FLORIDA
1,2016.09.18.b,2016,USA,FLORIDA
2,2016.09.18.a,2016,USA,FLORIDA
3,2016.09.17,2016,AUSTRALIA,VICTORIA
4,2016.09.15,2016,AUSTRALIA,VICTORIA
5,2016.09.15.R,2016,AUSTRALIA,WESTERN_AUSTRALIA
6,2016.09.11,2016,USA,FLORIDA
7,2016.09.07,2016,USA,HAWAII
8,2016.09.06,2016,NEW_CALEDONIA,NORTH_PROVINCE
9,2016.09.05.b,2016,USA,SOUTH_CAROLINA


In [39]:
# Muestro los nombres de las columnas de mi dataset, para ver si me puedo deshacer de alguna que no 
# tenga relevancia para mi análisis.

print(dataset.columns.values)

['Case Number' 'Date' 'Year' 'Type' 'Country' 'Area' 'Location' 'Activity'
 'Name' 'Sex ' 'Age' 'Injury' 'Fatal (Y/N)' 'Time' 'Species '
 'Investigator or Source' 'pdf' 'href formula' 'href' 'Case Number.1'
 'Case Number.2' 'original order' 'Unnamed: 22' 'Unnamed: 23']


In [40]:
# Hago una lista de las columnas de las que me quiero deshacer. No me deshago de "Case Number" porque lo utilizaré
# después para eliminar duplicados.

unrelevant_cols = ["Date", "Type", "Location", "Activity", "Sex ", "Age", "Injury", "Fatal (Y/N)", 
                   "Time", "Species ",  "Name", "Investigator or Source", "pdf", "href formula", "href", 
                   "Case Number.1", "Case Number.2", "original order", "Unnamed: 22", "Unnamed: 23"]

In [41]:
# Me deshago de ellas con un drop.

dataset = dataset.drop(unrelevant_cols, axis=1)

In [42]:
# Muestro el head de mi dataset actualizado.

display(dataset.head(8))

,Case Number,Year,Country,Area
0,2016.09.18.c,2016,USA,Florida
1,2016.09.18.b,2016,USA,Florida
2,2016.09.18.a,2016,USA,Florida
3,2016.09.17,2016,AUSTRALIA,Victoria
4,2016.09.15,2016,AUSTRALIA,Victoria
5,2016.09.15.R,2016,AUSTRALIA,Western Australia
6,2016.09.11,2016,USA,Florida
7,2016.09.07,2016,USA,Hawaii


In [43]:
# Compruebo cuántos valores nulos tengo en cada columna. 

null_cols = dataset.isnull().sum()

null_cols[null_cols > 0]

Country     43
Area       402
dtype: int64

In [44]:
# Si los registros cuyo valor en Country es null no tienen valor null en Area, puedo detectar a qué océano 
# pertenece el registro. Sin embargo, si la mayoría de los resgistros cuyo valor en Country tienen valor null
# en Area también, me puedo deshacer directamente de los resgistros cuyo valor en Country es null, para simplificar.

# Miro a ver cuántos registros que tengan valor null en Country, lo tienen también en Area.



null_countries = dataset[dataset.Country.isnull() & dataset.Area.isnull()]

print(len(null_countries.index))

30


In [45]:
# Para 43 registros cuyo valor en Country es null, 30 son null también en Area (el 70%). Por tanto, byebye.

dataset = dataset[dataset.Country.notnull()]

In [46]:
# Voy a tener que usar los años como enteros para medir el paso en el tiempo, así que compruebo que el type de Year 
# sea integer:

dataset.dtypes

Case Number    object
Year            int64
Country        object
Area           object
dtype: object

In [47]:
# Para poder categorizar los países, tengo que limpiar los strings de caracteres especiales. 

# Primero cambio todos los espacios por "_", para no cargarme los separadores de strings:

dataset['Country'] = dataset['Country'].str.replace(' ', '_')
dataset['Area'] = dataset['Area'].str.replace(' ', '_')

In [48]:
# Y ahora borro todos los caracteres especiales:

dataset['Country'] = dataset['Country'].str.replace('\W', '')
dataset['Area'] = dataset['Area'].str.replace('\W', '')

In [49]:
# Compruebo a ver si todos los valores de Country y de Area son strings

for Country in dataset:
    for r in dataset["Country"]:
        if type(r) != str:
            print("C:", r)
            
for Area in dataset:
    for r in dataset["Area"]:
        if type(r) != str:
            print("A:", r)

A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan
A: nan

In [50]:
# Tengo que convertir todos los valores de Area a string, porque en la siguiente operación necesito todo strings.

dataset["Area"] = dataset["Area"].astype(str)

In [51]:
# Si había algún espacio al final o al principio, tengo que deshacerme ahora de los "_" del final/principio:

for Country in dataset:
    for r in dataset["Country"]:
        if r[-1] == "_":
            r_no_ = r.replace(r, r[:-2])
            dataset['Country'] = dataset['Country'].str.replace(r, r_no_)
        if r[0] == "_":
            r_no__ = r.replace(r, r[1:])
            dataset['Country'] = dataset['Country'].str.replace(r, r_no__)
            
for Area in dataset:
    for r in dataset["Area"]:
        if r[-1] == "_":
            r_no_ = r.replace(r, r[:-2])
            dataset['Area'] = dataset['Area'].str.replace(r, r_no_)
        if r[0] == "_":
            r_no__ = r.replace(r, r[1:])
            dataset['Area'] = dataset['Area'].str.replace(r, r_no__)

In [52]:
# Si había dos espacios seguidos entre los strings de los países, tengo que cambiar "__" por "_":

for Country in dataset:
    for r in dataset["Country"]:
        if "__" in r:
            rno__ = r.replace("__", "_")
            dataset['Country'] = dataset['Country'].str.replace(r, rno__)
            
for Area in dataset:
    for r in dataset["Area"]:
        if "__" in r:
            rno__ = r.replace("__", "_")
            dataset['Area'] = dataset['Area'].str.replace(r, rno__)

In [53]:
# Convierto todos los strings a mayúsculas

for Country in dataset:
    for c in dataset["Country"]:
        C = c.upper()
        dataset['Country'] = dataset['Country'].str.replace(c, C)
            
for Area in dataset:
    for a in dataset["Area"]:
        A = a.upper()
        dataset['Area'] = dataset['Area'].str.replace(a, A)

In [54]:
# Ahora con Country y Area limpios, tengo que limpiar Year. Compruebo a ver si hay algún valor que no sea integer

for Year in dataset:
    for r in dataset["Year"]:
        if type(r) != int:
            print(r)

In [55]:
# En segundo lugar, compruebo a ver si hay algún valor cuyo len sea mayor que 5, que sea igual a 0, o que sea negativo

for Year in dataset:
    for r in dataset["Year"]:
        if len(str(r)) > 4:
            print(r)
        if r <= 0:
            print(r)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [56]:
# Evidentemente, esto no significa que los ataques tuvieron lugar en el año 0, sino que no se ha registrado el año.
# Por lo tanto, sin año no me sirve el registro, así que me deshago de estos registros.

for Year in dataset:
    for y in dataset["Year"]:
        if y == 0:
            dataset = dataset[dataset.Year != 0]

In [57]:
# Ahora que ya tengo el dataset aparentemente limpio, voy a empezar a clasificar los registros por océano.

# Hay países que tienen costa en distintos océanos, así que voy a comprobar si mis registros incluyen estos países,
# y si considero relevante la división del país según su costa

# Voy a mostrar una tupla de los países que incluye mi dataset, creando primero una lista y convirtiéndola en tupla.

list_countries = []

for countries in dataset["Country"]:
    list_countries.append(countries)
    
tup_countries = set(list_countries)

print(len(tup_countries))
print(tup_countries)

189
{'MARSHALL_ISLANDS', 'CARIBBEAN_SEA', 'SUDAN', 'PACIFIC_OCEA', 'LIBYA', 'MAYOTTE', 'AZORES', 'ALGERIA', 'CAPE_VERDE', 'BERMUDA', 'ARGENTINA', 'NEW_GUINEA', 'SOLOMON_ISLANDS_VANUATU', 'SOUTH_CHINA_SEA', 'BARBADOS', 'SOUTHWEST_PACIFIC_OCEAN', 'DOMINICAN_REPUBLIC', 'SYRIA', 'PERSIAN_GULF', 'MALAYSIA', 'CURACAO', 'SOLOMON_ISLANDS', 'COSTA_RICA', 'NEW_CALEDONIA', 'SAN_DOMINGO', 'PUERTO_RICO', 'SLOVENIA', 'PORTUGAL', 'AUSTRALIA', 'THE_BALKANS', 'GULF_OF_ADEN', 'CROATIA', 'IRAQ', 'SOUTH_PACIFIC_OCEAN', 'CAYMAN_ISLANDS', 'RED_SEA_INDIAN_OCEAN', 'ECUADOR', 'NORTHERN_MARIANA_ISLANDS', 'DIEGO_GARCIA', 'BRAZIL', 'VIETNAM', 'EQUATORIAL_GUINEA_CAMEROON', 'UNITED_ARAB_EMIRATES', 'NICARAGUA', 'GUAM', 'PHILIPPINES', 'WESTERN_SAMOA', 'SEYCHELLES', 'TUNISIA', 'FALKLAND_ISLANDS', 'ICELAND', 'KUWAIT', 'NIGERIA', 'SPAIN', 'INDIA', 'FIJI', 'JOHNSTON_ISLAND', 'NORTHERN_ARABIAN_SEA', 'CENTRAL_PACIFIC', 'GRENADA', 'NORTH_ATLANTIC_OCEA', 'NEW_ZEALAND', 'SRI_LANKA', 'GABON', 'IRAN', 'OCEAN', 'NETHERLANDS_ANTI

In [58]:
# Limpio los mispellings

for Country in dataset:
    for c in dataset["Country"]:
        if c == "EGYP":
            ce = "EGYPT"
            dataset["Country"] = dataset["Country"].str.replace(c, ce)
            
for Country in dataset:
    for c in dataset["Country"]:
        if c == "EGYP_ISRAEL":
            ce = "EGYPT"
            dataset["Country"] = dataset["Country"].str.replace(c, ce)
            


In [98]:
for c in tup_countries:
    if c[0] == "C":
        print(c)

CARIBBEAN_SEA
CAPE_VERDE
CURACAO
COSTA_RICA
CROATIA
CAYMAN_ISLANDS
CENTRAL_PACIFIC
CHILE
COAST_OF_AFRICA
CANADA
COOK_ISLANDS
CHINA
CYPRUS
CUBA
COLUMBIA
CRETE


In [103]:
filtered = dataset[(dataset['Country']== 'CHILE') &
                  (dataset["Area"] != "BOCAS") &
                  (dataset["Area"] != "CARIBBEAN_SEA") &
                  (dataset["Area"] != "CARIBBEAN_COAST") &
                  (dataset["Area"] != "CABO_SAN_LUCAS") &
                  (dataset["Area"] != "COLÓN_PROVINCE") &
                  (dataset["Area"] != "SAN_BLAS_ISLANDS") &
                  (dataset["Area"] != "VERACRUZ") &
                  (dataset["Area"] != "TAMAULIPAS") &
                  (dataset["Area"] != "SINALOA")]



display(filtered)

,Case Number,Year,Country,Area
592,2012.03.01,2012,CHILE,ANTOFAGASTA_PROVINCE
2219,1994.03.23.b,1994,CHILE,NAN
2220,1994.03.23.a,1994,CHILE,NAN
2478,1988.12.15,1988,CHILE,VALPARISO_PROVINCE
2812,1981.03.04,1981,CHILE,COQUIMBO
2853,1980.01.05,1980,CHILE,LOS_VILOS
3511,1963.09.29,1963,CHILE,NAN
4718,1934.02.22,1934,CHILE,ELQUI_PROVINCE


In [ ]:
# Para simplificar mi diccionario, me voy a deshacer de todos aquellos registros cuyos países no estén en el top 50

dataset["Country"].value_counts()

In [ ]:
# Para ello, voy a crear una columna "Counts" que indique el count del país de cada registro:

dataset['Counts'] = dataset.groupby(['Country'])['Area'].transform('count')

In [ ]:
# Y ahora voy a eliminar todos los registros cuyo valor de Counts sea menor que 20

dataset = dataset[dataset.Counts >= 35]

In [ ]:
# Ya no me sirve de nada la columna Counts, así que me la bajo también

dataset = dataset.drop("Counts", axis = 1)

In [ ]:
display(dataset)

In [ ]:
dataset["Area"].value_counts()